# HOW TO ADD NEW SPECIES TO `EXOSPHID`

## Main Concept

The Exospheric Photolysis model is configured to support only a limited number of atomic and molecular species by default. These include:

- Water ($H_2O$)  
- Hydroxyl ($OH$)  
- Molecular Hydrogen ($H_2$)  
- Atomic Hydrogen ($H$)  
- Ionized (negative) Hydrogen ($H^{-}$)  
- Hydroperoxyl ($HO_2$)  
- Hydrogen Peroxide ($H_2O_2$)  
- Helium ($He$)  
- Neon ($Ne$)  
- Argon ($Ar$)

So far, the model has been applied primarily for investigations involving the Moon. This explains the focus on hydrogen- and oxygen-bearing species (for studying the lunar water cycle) and noble gases such as Neon and Helium, which are the main constituents of the Moon's exosphere.  

Technically, any species of interest could be added to the exospheric model for any exospheric investigations or for studying potential exospheres of other celestial bodies. In this notebook, two methods are suggested for adding new species.

### Function to print fluxes in correct format

Then we can directly paste them in the `solar_database.jl`script

In [31]:
using Printf

function print_flux(flux_data)

    arr = Float32.(Tuple(flux_data))

    print("(")  # start of tuple

    # Print first 4 elements
    for i in 1:min(4, length(arr))
        @printf("%.8e", arr[i])
        if i != min(4, length(arr))
            print(", ")
        end
    end
    if length(arr) > 4
        println(",")  # end first line with comma
    else
        println(")")  # only 4 elements, close tuple
    end

    # Print remaining elements 16 per line
    for i in 5:length(arr)
        @printf("%.8e", arr[i])
        # comma if not last element
        if (i-4) % 16 != 0
            print(", ")
        end
        # newline every 16 elements
        if (i-4) % 16 == 0 && i != length(arr)
            println(",")  # line break with comma
        end
    end

    println(")")  # close tuple at the very end
end

print_flux (generic function with 1 method)

## Example: Helium

In [32]:
if !isdefined(Main, :EXOSPHID)
    include(joinpath(@__DIR__, "..", "src", "EXOSPHID.jl"))
    using .EXOSPHID
end

**Methodology**

- Literature sources will usually provide database for wavelength dependent absorption cross-section data, which will be different for every species.
- We already have solar flux values at 1 A.U. given by W.F. Huebner (1992) in photons $cm^{-2}$ $s^{-1}$.
- First, we shall obtain the species dependent fluxes expressed in number of photons per unit time for the new species we want to add to `EXOSPHID`
- To do that, we will need to reference the data to the reference wavelength bins (`lambda_phi`) provided by W.F. Huebner (1992)
- Then, we shall do the numerical product of both quantities
- Additionnally, we may calculate the photodestruction rate 

#### Solar flux expressed in $cm^{-2} s^{-1}$ per wavelength bin from Huebner (2015)

In [33]:
# Wavelength values
lambda_phi = [
    1, 2, 4, 6, 8, 10, 40, 50, 60, 70, 80, 90, 100, 110, 120, 128, 153, 176, 205, 231, 270, 280, 300, 320, 340, 360, 370, 400, 430, 460, 480, 500, 
    520, 540, 560, 580, 600, 620, 630, 680, 710, 740, 770, 800, 830, 860, 890, 911, 920, 930, 940, 950, 960, 970, 980, 995, 1007, 1017, 1027, 1035, 1045, 1057, 1070, 1080, 
    1090, 1100, 1110, 1120, 1130, 1140, 1150, 1157, 1163, 1170, 1176, 1183, 1190, 1198, 1205, 1212, 1220, 1227, 1235, 1242, 1250, 1258, 1266, 1274, 1282, 1290, 1299, 1307, 1316, 1324, 1333, 1342,
    1351, 1360, 1370, 1379, 1389, 1408, 1428, 1449, 1470, 1492, 1515, 1538, 1562, 1587, 1613, 1639, 1667, 1695, 1724, 1739, 1754, 1770, 1786, 1802, 1818, 1835, 1852, 1869, 1887, 1905, 1923, 1942, 
    1961, 1980, 2000, 2020, 2041, 2062, 2083, 2105, 2128, 2150, 2174, 2198, 2222, 2247, 2273, 2299, 2326, 2353, 2381, 2410, 2439, 2469, 2500, 2532, 2564, 2597, 2632, 2667, 2703, 2740, 2778, 2817,
    2857, 2899, 2941, 2985, 3030, 3077, 3125, 3175, 3225, 3275, 3325, 3375, 3425, 3475, 3525, 3575, 3625, 3675, 3725, 3775, 3825, 3875, 3925, 3975, 4025, 4075, 4125, 4175, 4225, 4275, 4325, 4375, 4425, 
    4475, 4525, 4575, 4625, 4675, 4725, 4775, 4825, 4875, 4925, 4975, 5025, 5075, 5125, 5175, 5225, 5275, 5325, 5375, 5425, 5475, 5525, 5575, 5625, 5675, 5725, 5775, 5825, 5875, 5925, 5975, 6025, 6075, 
    6125, 6175, 6225, 6275, 6325, 6375, 6425, 6475, 6550, 6650, 6750, 6850, 6950, 7050, 7150, 7250, 7350, 7450, 7550, 7650, 7750, 7850, 7950, 8050, 8150, 8250, 8350, 8450, 8550, 8650, 8750, 8850, 8950, 
    9050, 9150, 9250, 9350, 9450, 9550, 9650, 9750, 9850, 9950, 10150, 10750, 11250, 11750, 12250, 12750, 13250, 13750, 14250, 14750, 15250, 15750, 16250, 16750, 17250, 17750, 18250, 18750, 19250, 19750, 20375, 21500, 22500, 
    23500, 24500, 25500, 26500, 27500, 28500, 29500, 30500, 31500, 32500, 33500, 34500, 35500, 36500, 37500, 38500, 39500, 40500, 41500, 42500, 43500, 44500, 45500, 46500, 47500, 48500, 49500, 52750, 65000, 75000, 85000, 95000
]

# Flux in cm^-2 s^-1 bin^-1 (scientific notation) from Huebner (1992)
phi = [
    0.0, 3.0e1, 2.5e3, 2.8e4, 1.8e5, 4.0e6, 4.7e7, 8.3e7, 1.03e8, 9.4e7, 1.2e8, 9.9e7, 5.6e7, 2.5e7, 1.2e7, 3.07e8, 9.0e8, 3.7e9, 1.4e9, 2.65e9, 4.5e8, 1.54e9, 5.84e9, 1.16e9, 4.5e8, 1.31e9, 1.77e8, 2.76e8, 1.77e8, 2.12e8, 4.32e8, 6.32e8, 
    2.42e8, 3.62e8, 5.17e7, 1.02e9, 5.52e8, 1.2e9, 1.15e8, 3.39e8, 1.59e8, 5.19e8, 1.28e9, 8.07e8, 1.05e9, 3.73e9, 3.34e9, 9.39e7, 2.34e8, 5.84e8, 5.94e8, 1.04e8, 1.04e8, 5.3e9, 7.56e8, 1.25e8, 1.04e8, 3.6e9, 2.46e9, 1.83e9, 1.57e8, 1.7e8, 1.31e8, 7.21e8, 
    1.31e8, 1.31e8, 1.31e8, 6.21e8, 1.31e8, 1.31e8, 9.15e7, 7.84e7, 1.03e8, 2.66e8, 1.12e8, 1.24e8, 1.82e8, 1.9e8, 7.4e8, 3.02e11, 3.67e9, 1.36e9, 1.61e9, 1.32e9, 1.41e9, 3.11e9, 1.06e9, 1.37e9, 1.02e9, 1.14e9, 7.29e9, 2.2e9, 1.59e9, 2.21e9, 1.24e9, 1.99e9, 
    3.09e9, 2.57e9, 2.74e9, 3.1e9, 7.6e9, 1.01e10, 1.3e10, 1.82e10, 1.33e10, 1.66e10, 2.9e10, 3.6e10, 4.75e10, 6.4e10, 5.49e10, 1.19e11, 1.76e11, 2.32e11, 1.44e11, 1.83e11, 1.34e11, 2.62e11, 2.88e11, 3.14e11, 3.81e11, 4.43e11, 4.95e11, 5.94e11, 6.59e11, 7.26e11, 9.85e11, 1.27e12, 
    1.01e12, 1.2e12, 1.44e12, 1.8e12, 2.08e12, 2.45e12, 5.09e12, 7.12e12, 9.23e12, 8.42e12, 1.2e13, 1.22e13, 1.77e13, 1.6e13, 1.96e13, 2.4e13, 2.25e13, 2.21e13, 2.32e13, 2.5e13, 2.73e13, 2.88e13, 3.02e13, 3.97e13, 7.13e13, 4.37e13, 1.12e14, 1.25e14, 1.16e14, 1.19e14, 1.38e14, 1.70e14,
    2.46e14, 3.90e14, 3.99e14, 3.86e14, 5.08e14, 5.92e14, 6.05e14, 6.94e14, 8.12e14, 9.71e14, 8.97e14, 9.44e14, 1.01e15, 1.03e15, 1.03e15, 1.04e15, 1.18e15, 1.23e15, 1.24e15, 1.17e15, 1.11e15, 1.09e15, 1.19e15, 1.54e15, 1.90e15, 1.99e15, 1.99e15, 2.02e15, 2.01e15, 1.94e15, 1.98e15, 2.25e15, 2.39e15, 
    2.48e15, 2.49e15, 2.48e15, 2.50e15, 2.55e15, 2.61e15, 2.59e15, 2.46e15, 2.44e15, 2.53e15, 2.48e15, 2.49e15, 2.50e15, 2.43e15, 2.43e15, 2.52e15, 2.58e15, 2.64e15, 2.67e15, 2.70e15, 2.68e15, 2.66e15, 2.66e15, 2.67e15, 2.67e15, 2.69e15, 2.71e15, 2.71e15, 2.71e15, 2.72e15, 2.72e15, 2.71e15, 2.70e15,
    2.70e15, 2.70e15, 2.69e15, 2.68e15, 2.67e15, 2.66e15, 2.65e15, 3.95e15, 5.22e15, 5.18e15, 5.14e15, 5.09e15, 5.04e15, 4.99e15, 4.94e15, 4.90e15, 4.83e15, 4.80e15, 4.67e15, 4.59e15, 4.68e15, 4.54e15, 4.60e15, 4.53e15, 4.41e15, 4.34e15, 4.31e15, 4.03e15, 4.33e15, 4.25e15, 4.27e15, 4.23e15, 4.13e15, 
    4.01e15, 3.89e15, 3.88e15, 3.78e15, 3.76e15, 3.72e15, 3.72e15, 3.79e15, 3.79e15, 7.49e15, 2.11e16, 1.68e16, 1.60e16, 1.50e16, 1.48e16, 1.43e16, 1.32e16, 1.25e16, 1.16e16, 1.12e16, 1.07e16, 9.95e15, 9.72e15, 9.20e15, 8.24e15, 7.70e15, 6.94e15, 6.51e15, 6.19e15, 7.46e15, 1.11e16, 8.28e15, 7.33e15, 
    6.83e15, 6.07e15, 5.50e15, 4.96e15, 4.51e15, 4.09e15, 3.74e15, 3.40e15, 3.18e15, 2.87e15, 2.70e15, 2.47e15, 2.31e15, 2.14e15, 2.01e15, 1.87e15, 1.71e15, 1.60e15, 1.48e15, 1.41e15, 1.33e15, 1.25e15, 1.16e15, 1.06e15, 9.67e14, 9.25e14, 2.84e15, 6.74e15, 3.35e15, 2.22e15, 1.59e15, 4.53e15
]

324-element Vector{Float64}:
      0.0
     30.0
   2500.0
  28000.0
 180000.0
      4.0e6
      4.7e7
      8.3e7
      1.03e8
      9.4e7
      ⋮
      1.06e15
      9.67e14
      9.25e14
      2.84e15
      6.74e15
      3.35e15
      2.22e15
      1.59e15
      4.53e15

#### Solar flux ratio between active and quiet sun conditions from Huebner (1992)

Extracted from Figure 3 with PlotDigitizer: https://plotdigitizer.com/

**NOTE:**
- To calculate the fluxes for active Sun conditions, we have the ration between active and quiet fluxes provided by Huebner (1992) as a plot
- As the numerical database was not found, the values from the plot were exctracted with a plot digitizer tool

In [34]:
# Wavelength vector
lambda_active = [0.00, 28.72, 53.29, 59.16, 59.16, 71.54, 71.54, 74.44, 88.08, 89.99, 
115.77, 115.77, 125.25, 125.25, 144.88, 144.88, 148.61, 148.61, 165.67, 176.91, 
223.91, 231.74, 272.68, 283.07, 298.20, 298.20, 300.99, 342.41, 344.49, 365.70, 
365.70, 366.69, 424.97, 426.87, 479.82, 480.91, 501.09, 534.21, 541.24, 580.65, 
585.59, 606.49, 609.04, 630.15, 633.66, 678.40, 683.51, 712.99, 714.07, 739.78, 
741.06, 769.47, 785.37, 796.68, 836.69, 922.60, 922.60, 961.07, 963.04, 970.42, 
1022.55, 1022.55, 1028.79, 1073.51, 1074.25, 1126.39, 1131.24, 1175.13, 1180.71, 
1184.81, 1224.35, 1271.98, 1278.65, 1300.04, 1318.69, 1326.50, 1357.78, 1364.96, 
1368.52, 1384.26, 1384.26, 1396.93, 1480.30, 1489.65, 1529.55, 1557.35, 1581.17, 
1610.21, 1615.05, 1684.84, 1687.57, 1786.33, 1918.57, 1938.65, 2034.93, 2049.99, 
2080.65, 2116.71, 2138.45, 2462.53, 2500.0, 95000.0]


# Ratio of flux active/quiet sun from Huebner (1992). Taken from Graph
ratio = [6.37, 6.37, 6.37, 2.50, 2.50, 2.32, 2.32, 2.48, 2.12, 2.31, 
2.35, 2.35, 2.58, 2.58, 3.60, 3.60, 4.51, 4.51, 4.51, 5.38, 
2.11, 5.37, 2.54, 2.10, 2.54, 2.54, 2.95, 2.14, 2.95, 2.83, 
2.83, 2.14, 2.41, 2.40, 2.40, 2.30, 2.27, 2.21, 2.01, 1.88, 
2.00, 1.79, 1.88, 1.94, 1.79, 1.94, 2.05, 2.18, 2.04, 1.93, 
2.18, 2.48, 1.91, 2.49, 2.41, 2.41, 1.97, 1.96, 2.14, 2.35, 2.36, 
1.98, 1.72, 1.72, 1.84, 1.84, 1.92, 1.84, 1.93, 2.34, 2.15, 
2.11, 1.87, 1.86, 1.88, 2.07, 1.92, 2.06, 1.99, 1.89, 1.89, 
1.65, 1.69, 1.55, 1.49, 1.48, 1.43, 1.42, 1.37, 1.26, 1.21, 
1.16, 1.15, 1.13, 1.08, 1.09, 1.05, 1.05, 1.03, 1.03, 1.01, 1.0]

102-element Vector{Float64}:
 6.37
 6.37
 6.37
 2.5
 2.5
 2.32
 2.32
 2.48
 2.12
 2.31
 ⋮
 1.13
 1.08
 1.09
 1.05
 1.05
 1.03
 1.03
 1.01
 1.0

#### Interpolated ratios for the wavelength bins given in Huebner (1992)

In [35]:
using Interpolations

In [36]:
Interpolations.deduplicate_knots!(lambda_active)

itp = interpolate((lambda_active,), ratio, Gridded(Linear()))
ratio_interp = itp(lambda_phi)

324-element Vector{Float64}:
 6.369999999999999
 6.37
 6.37
 6.370000000000001
 6.37
 6.369999999999999
 6.37
 6.37
 2.4877867528271405
 2.3423909531502423
 ⋮
 1.0052432432432432
 1.0051351351351352
 1.005027027027027
 1.004918918918919
 1.0045675675675676
 1.0032432432432432
 1.0021621621621621
 1.001081081081081
 1.0

### Cross sectional database is available per wavelength value

Example: Helium cross-sectional data from  Marr, West (1976)

In [37]:
He_CS_data = [(505.0, 1e-20),
    (504.0, 7.56), (500.0, 7.46), (495.0, 7.33), (490.0, 7.19), (485.0, 7.06), (480.0, 6.94),
    (475.0, 6.81), (470.0, 6.68), (465.0, 6.55), (460.0, 6.43), (455.0, 6.30),
    (450.0, 6.18), (445.0, 6.05), (440.0, 5.93), (435.0, 5.81), (430.0, 5.69),
    (425.0, 5.57), (420.0, 5.45), (415.0, 5.33), (410.0, 5.21), (405.0, 5.10),
    (400.0, 4.98), (395.0, 4.87), (390.0, 4.76), (385.0, 4.64), (380.0, 4.53),
    (375.0, 4.42), (370.0, 4.31), (365.0, 4.20), (360.0, 4.09), (355.0, 4.00),
    (350.0, 3.88), (345.0, 3.78), (340.0, 3.68), (335.0, 3.57), (330.0, 3.47),
    (325.0, 3.37), (320.0, 3.27), (315.0, 3.18), (310.0, 3.08), (305.0, 2.98),
    
    (300.0, 2.89), (295.0, 2.80), (290.0, 2.70), (285.0, 2.61), (280.0, 2.52),
    (275.0, 2.44), (270.0, 2.35), (265.0, 2.26), (260.0, 2.18), (255.0, 2.10),
    (250.0, 2.02), (245.0, 1.94), (240.0, 1.86), (235.0, 1.78), (230.0, 1.70),
    (225.0, 1.63), (220.0, 1.55), (215.0, 1.48), (210.0, 1.41), (205.0, 1.34),
    (200.0, 1.28), (195.0, 1.21), (190.0, 1.14), (185.0, 1.08), (180.0, 1.02),
    (175.0, 0.961), (170.0, 0.903), (165.0, 0.847), (160.0, 0.792), (155.0, 0.738),
    (150.0, 0.687), (145.0, 0.637), (140.0, 0.588), (135.0, 0.542), (130.0, 0.497),
    (125.0, 0.454), (120.0, 0.412), (115.0, 0.373), (110.0, 0.335), (105.0, 0.299), (100.0, 0.265), 
    
    (95.0, 0.233), (90.0, 0.202), ( 85.0, 0.174), ( 80.0, 0.147), ( 75.0, 0.123), 
    (70.0, 0.100), (65.0, 0.0795), ( 60.0, 0.0609),( 55.0, 0.0443), ( 50.0, 0.0315), 
    (48.0, 0.0282), (46.0, 0.0250),  (44.0, 0.0220), (42.0, 0.0193), (40.0, 0.0168), 
    (38.0, 0.0145), (36.0, 0.0124), (34.0, 0.0105), (32.0, 0.00885), (30.0, 0.00736), 
    (28.0, 0.00604), (26.0, 0.00489), (24.0, 0.00389), (22.0, 0.00303), (20.0, 0.00231), 
    (19.0, 0.00199), (18.0, 0.00171), (17.0, 0.00145), (16.0, 0.00122),(15.0, 0.00101), 
    (14.0, 0.000832), (13.0, 0.000673), (12.0, 0.000535), (11.0, 0.000417), (10.0, 0.000318), 
    (9.5, 0.000274), (9.0, 0.000235), (8.5, 0.000200), (8.0, 0.000168), (7.5, 0.000139), (7.0, 0.000115), 
    
    (6.5, 0.000093), (6.0, 0.000074), (5.5, 0.000057), (5.0, 0.000044), (4.5, 0.000031), 
    (4.0, 0.000023), (3.5, 0.000016), (3.0, 0.000010), (2.5, 0.000006), (2.0, 0.000003), (1.5, 0.000001), (0.0, 1e-20)
]

# Separate into two Python lists: wavelengths and cross sections
wvl = reverse([r[1] for r in He_CS_data])
cs  = reverse([r[2] for r in He_CS_data])

136-element Vector{Float64}:
 1.0e-20
 1.0e-6
 3.0e-6
 6.0e-6
 1.0e-5
 1.6e-5
 2.3e-5
 3.1e-5
 4.4e-5
 5.7e-5
 ⋮
 6.68
 6.81
 6.94
 7.06
 7.19
 7.33
 7.46
 7.56
 1.0e-20

#### Interpolate cross sectional data to have it in the same reference wvl as the solar flux

In [38]:
itp = LinearInterpolation(wvl, cs, extrapolation_bc=Flat())  
cs_interp = itp.(lambda_phi)

324-element Vector{Float64}:
 6.666666666666699e-7
 3.0e-6
 2.3e-5
 7.4e-5
 0.000168
 0.000318
 0.0168
 0.0315
 0.0609
 0.1
 ⋮
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20
 1.0e-20

#### Quiet Sun

Calculate flux in photons/second and print photodestruction rate

In [39]:
He_flux_quiet = cs_interp .* 1e-18 .* phi

println("Photodestruction rate: ", sum(He_flux_quiet))

Photodestruction rate: 5.5744213704376146e-8


Print flux and copy it to `solar_database.jl` accordingly

In [40]:
print_flux(He_flux_quiet)

(0.00000000e+00, 8.99999984e-23, 5.74999998e-20, 2.07199996e-18,
3.02399994e-17, 1.27199996e-15, 7.89599993e-13, 2.61449998e-12, 6.27269981e-12, 9.39999988e-12, 1.76400006e-11, 1.99979998e-11, 1.48400008e-11, 8.37500035e-12, 4.94400007e-12, 1.47298604e-10, 6.45839993e-10, 3.59935992e-09, 1.87600002e-09, 4.54739979e-09,
1.05749998e-09, 3.88080013e-09, 1.68775998e-08, 3.79319998e-09, 1.65600000e-09, 5.35789990e-09, 7.62869989e-10, 1.37447997e-09, 1.00713005e-09, 1.36316003e-09, 2.99808001e-09, 4.71472017e-09, 2.42000000e-30, 3.62000012e-30, 5.17000001e-31, 1.02000003e-29,
5.51999997e-30, 1.20000000e-29, 1.15000004e-30, 3.39000007e-30, 1.58999999e-30, 5.19000015e-30, 1.27999997e-29, 8.07000006e-30, 1.05000000e-29, 3.72999998e-29, 3.34000007e-29, 9.39000028e-31, 2.33999999e-30, 5.84000000e-30, 5.94000015e-30, 1.04000001e-30,
1.04000001e-30, 5.29999972e-29, 7.56000026e-30, 1.25000000e-30, 1.04000001e-30, 3.60000001e-29, 2.45999995e-29, 1.82999998e-29, 1.57000004e-30, 1.69999993e-30, 1.30999

Calculate the normalized flux values when considering the full electromagnetic spectrum. This is the weight of a certain photon having a certain wavelength/energy per unit time. 
Print normalized fluxes and modify `solar_database.jl` accordingly

In [41]:
print_flux(EXOSPHID.normalize_flux_distribution(lambda_phi, He_flux_quiet, (0.0, 95000))[2])

(0.00000000e+00, 1.61451732e-15, 1.03149715e-12, 3.71697777e-11,
5.42477840e-10, 2.28185115e-08, 1.41646988e-05, 4.69017286e-05, 1.12526475e-04, 1.68627361e-04, 3.16445425e-04, 3.58745747e-04, 2.66215997e-04, 1.50239823e-04, 8.86908238e-05, 2.64240173e-03, 1.15857767e-02, 6.45692125e-02, 3.36537175e-02, 8.15761760e-02,
1.89705789e-02, 6.96179941e-02, 3.02768648e-01, 6.80465251e-02, 2.97071189e-02, 9.61158052e-02, 1.36851873e-02, 2.46569086e-02, 1.80669893e-02, 2.44538393e-02, 5.37828021e-02, 8.45777541e-02, 4.34125776e-23, 6.49394778e-23, 9.27450513e-24, 1.82978647e-22,
9.90237341e-23, 2.15268983e-22, 2.06299449e-23, 6.08134891e-23, 2.85231404e-23, 9.31038379e-23, 2.29620244e-22, 1.44768396e-22, 1.88360357e-22, 6.69127774e-22, 5.99165328e-22, 1.68447978e-23, 4.19774502e-23, 1.04764237e-22, 1.06558145e-22, 1.86566456e-23,
1.86566456e-23, 9.50771312e-22, 1.35619465e-22, 2.24238524e-23, 1.86566456e-23, 6.45806975e-22, 4.41301412e-22, 3.28285185e-22, 2.81643601e-23, 3.04964381e-23, 2.35001

#### Active Sun

Calculate flux in photons/second and print photodestruction rate

In [42]:
He_flux_active = cs_interp .* 1e-18 .* phi .* ratio_interp

print("Photodestruction rate:", sum(He_flux_active))


Photodestruction rate:1.6803216302077845e-7

Print flux and copy it to `solar_database.jl` accordingly

In [43]:
print_flux(He_flux_active)

(0.00000000e+00, 5.73299980e-22, 3.66275007e-19, 1.31986403e-17,
1.92628805e-16, 8.10264009e-15, 5.02975179e-12, 1.66543654e-11, 1.56051404e-11, 2.20184756e-11, 4.11586216e-11, 4.61956896e-11, 3.45108872e-11, 1.96062715e-11, 1.21257856e-11, 4.01078393e-10, 2.91273849e-09, 1.91110328e-08, 6.42652553e-09, 2.30184973e-08,
2.88195867e-09, 8.65422134e-09, 4.73335007e-08, 9.77979830e-09, 3.62188635e-09, 1.53356439e-08, 1.64424008e-09, 3.15349524e-09, 2.41711207e-09, 3.27158389e-09, 7.14588255e-09, 1.07100542e-08, 5.41049731e-30, 7.40390440e-30, 1.00717663e-30, 1.91978698e-29,
1.02407623e-29, 2.29338128e-29, 2.23050975e-30, 6.69335963e-30, 3.44523552e-30, 1.02397076e-29, 3.15007991e-29, 2.00407290e-29, 2.54454556e-29, 8.98930028e-29, 8.04940013e-29, 2.26299007e-30, 5.63940011e-30, 1.14935662e-29, 1.16749332e-29, 2.04139264e-30,
2.03868919e-30, 1.23916584e-28, 1.77798933e-29, 2.94339384e-30, 2.45129768e-30, 8.49216721e-29, 4.41467500e-29, 3.14759997e-29, 2.70039998e-30, 2.92400007e-30, 2.25319

Calculate the normalized flux values when considering the full electromagnetic spectrum. This is the weight of a certain photon having a certain wavelength/energy per unit time. 
Print normalized fluxes and modify `solar_spectrum.jl` accordingly

In [44]:
print_flux(EXOSPHID.normalize_flux_distribution(lambda_phi, He_flux_active, (0.0, 95000))[2])

(0.00000000e+00, 3.41184681e-15, 2.17979107e-12, 7.85482998e-11,
1.14638055e-09, 4.82207696e-08, 2.99332696e-05, 9.91141569e-05, 9.28699665e-05, 1.31037275e-04, 2.44944909e-04, 2.74921709e-04, 2.05382632e-04, 1.16681658e-04, 7.21634788e-05, 2.38691457e-03, 1.73344109e-02, 1.13734379e-01, 3.82458083e-02, 1.36988640e-01,
1.71512328e-02, 5.15033640e-02, 2.81693101e-01, 5.82019426e-02, 2.15547215e-02, 9.12661254e-02, 9.78527032e-03, 1.87672134e-02, 1.43848183e-02, 1.94699857e-02, 4.25268747e-02, 6.37381226e-02, 3.21991785e-23, 4.40624254e-23, 5.99395172e-24, 1.14251167e-22,
6.09452541e-23, 1.36484662e-22, 1.32743029e-23, 3.98338028e-23, 2.05034289e-23, 6.09389748e-23, 1.87468869e-22, 1.19267224e-22, 1.51432062e-22, 5.34974972e-22, 4.79039256e-22, 1.34676006e-23, 3.35614340e-23, 6.84009931e-23, 6.94803505e-23, 1.21488208e-23,
1.21327320e-23, 7.37457567e-22, 1.05812444e-22, 1.75168489e-23, 1.45882659e-23, 5.05389381e-22, 2.62727991e-22, 1.87321282e-22, 1.60707327e-23, 1.74014306e-23, 1.34093